<h1 align="center"> FYP Live Report </h1>


In [1]:
import py_dss_interface
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import csv
import folium
from   folium import plugins
import pandas as pd
import json
import vincent
import tkinter
import tkinter.messagebox
import customtkinter

Setting up path and object for DSS file.    
```python


In [2]:
dss_file = "..\\13Bus\IEEE13Nodeckt.dss"
dss = py_dss_interface.DSSDLL()

OpenDSS Started successfully! 
OpenDSS Version 9.4.0.1 (64-bit build); License Status: Open 




Compiling the script, in other words initializing it, which is then automatically executed due to "solve" command in the code

In [3]:
dss.text(f"compile [{dss_file}]")

''

Storing all PC and PD elements in the circuit in lists


In [4]:
transformers = []
caps = []
regulators = []
lines = []
loads = []

all_elements = dss.circuit_all_element_names()

for item in all_elements:
    if 'Transformer' in item:
        transformers.append(item)
    elif 'Capacitor' in item:
        caps.append(item)
    elif 'RegControl' in item:
        regulators.append(item)
    elif 'Line' in item:
        lines.append(item)
    elif 'Load' in item:
        loads.append(item)

bus_nodes = dss.circuit_all_node_names()
buses = []
for bus in bus_nodes:
    if bus.split(".")[0] not in buses:
        buses.append(bus.split(".")[0])

# this made manually
load_to_bus_mapping_ev = {
    "Load.633ev" : "633",
    "Load.632ev" : "632",
    "Load.680ev" : "680",
    "Load.684ev" : "684",
    "Load.634ev" : "634",
    "Load.671ev" : "671",
    "Load.645ev" : "645",
    "Load.646ev" : "646",
    "Load.692ev" : "692",
    "Load.675ev" : "675",
    "Load.611ev" : "611",
    "Load.652ev" : "652",
}

Function to plot before and after states of an element's parameters via bar graphs. Function to be depracated.

In [5]:
def show_before_and_after(dict, item, parameter):
    fig, ax = plt.subplots(figsize=(6,6))
    for i in range(0, len(dict[item][parameter][0]), 2):
        ax.arrow(0, 0, dict[item][parameter][0][i], dict[item][parameter][0][i+1], head_width=10, head_length=10, fc='blue', ec='blue')
        ax.arrow(0, 0, dict[item][parameter][1][i], dict[item][parameter][1][i+1], head_width=10, head_length=10, fc='red', ec='red')
    ax.grid(True)
    #ax.legend(['Before'])
    ax.legend(['Before', 'After'])
    ax.set_xlabel('Real Axis (A)')
    ax.set_ylabel('Imaginary Axis (A)')
    title = 'Complex ' + parameter + ' Before ' + 'and After'
    ax.set_title(title)
    plt.show(block=False)

In [6]:
def show_voltages(col):
    buses_list = dss.circuit_all_node_names()
    bus_voltages = dss.circuit_all_bus_vmag_pu()
    fig = plt.figure(figsize = (75, 5))
    plt.bar(buses_list, bus_voltages, color =col)
    plt.xlabel("Nodes")
    plt.ylabel("Voltage (reference to ground in V)")
    plt.title("Voltage at each node in the circuit")
    plt.show(block=False)
    return

In [7]:
def store_values():
    for load in loads:
        dss.circuit_set_active_element(load) 
        load_dict[load]["current"].append(dss.cktelement_currents())
        load_dict[load]["power"].append(dss.cktelement_powers())
        load_dict[load]["voltage"].append(dss.cktelement_voltages())
    for line in lines:
        dss.circuit_set_active_element(line) 
        line_dict[line]["current"].append(dss.cktelement_currents())
        line_dict[line]["power"].append(dss.cktelement_powers())
        line_dict[line]["voltage"].append(dss.cktelement_voltages())
    for cap in caps:
        dss.circuit_set_active_element(cap) 
        cap_dict[cap]["current"].append(dss.cktelement_currents())
        cap_dict[cap]["power"].append(dss.cktelement_powers())
        cap_dict[cap]["voltage"].append(dss.cktelement_voltages())
    for transformer in transformers:
        dss.circuit_set_active_element(transformer) 
        transformer_dict[transformer]["current"].append(dss.cktelement_currents())
        transformer_dict[transformer]["power"].append(dss.cktelement_powers())
        transformer_dict[transformer]["voltage"].append(dss.cktelement_voltages())
    for regulator in regulators:
        dss.circuit_set_active_element(regulator) 
        regulator_dict[regulator]["current"].append(dss.cktelement_currents())
        regulator_dict[regulator]["power"].append(dss.cktelement_powers())
        regulator_dict[regulator]["voltage"].append(dss.cktelement_voltages())


In [8]:
def change_loads(load, value, param):
    dss.text(f"compile [{dss_file}]")
    dss.text(f"solve")
    dss.loads_write_name(load.split(".")[-1])
    dss.loads_write_kw(value[0])
    dss.loads_write_kvar(value[1])
    dss.text(f"solve")

In [9]:
def make_dicts():
    load_dict = {}
    for load in loads:
        load_dict[load] = {"power":[], "current":[], "voltage":[]}
    line_dict = {}
    for line in lines:
        line_dict[line] = {"power":[], "current":[], "voltage":[]}
    cap_dict = {}
    for cap in caps:
        cap_dict[cap] = {"power":[], "current":[], "voltage":[]}
    transformer_dict = {}
    for transformer in transformers:
        transformer_dict[transformer] = {"power":[], "current":[],  "voltage":[]}
    regulator_dict = {}
    for regulator in regulators:
        regulator_dict[regulator] = {"power":[], "current":[],  "voltage":[]} 
    return load_dict, line_dict, cap_dict, transformer_dict, regulator_dict

In [10]:
def save_to_excel(log_data, excel_file, col_heads):
    df = pd.DataFrame(log_data, columns=col_heads)
    try:
        with pd.ExcelWriter(excel_file, mode='a') as writer:
            df.to_excel(writer, index=False, header=False, startrow=writer.sheets['index'].max_row)
    except FileNotFoundError:
        df.to_excel(excel_file, index=False, header=col_heads)

In [11]:
def compute_violations(ld, param, value):
    change_loads(ld, value, param)
    store_values()
    log_data=[]
    index=0
    for bus in buses:
        dss.circuit_set_active_bus(bus)
        if dss.bus_num_nodes()>1:
            for i in range(0,len(dss.bus_pu_vmag_angle()),2):
                if dss.bus_pu_vmag_angle()[i] > 1.05 or dss.bus_pu_vmag_angle()[i] < 0.95: 
                    if (dss.bus_pu_vmag_angle()[i] > 1.05): type = "overvoltage"
                    else: type = "undervoltage"
                    # [bus_name, bus_node, vmag_pu, all_connected_pce, all_connected_pde, over/under voltage, kW_at_failure, kvar_at_failure]
                    log_data.append([index, bus, i, dss.bus_pu_vmag_angle()[i], dss.bus_all_pce_active_bus(), dss.bus_all_pde_active_bus(), type, value[0], value[1]])
                    index+=1
        else:
            if dss.bus_pu_vmag_angle()[0] > 1.05 or dss.bus_pu_vmag_angle()[0] < 0.95:
                log_data.append([index, bus, i, dss.bus_pu_vmag_angle()[0], dss.bus_all_pce_active_bus(), dss.bus_all_pde_active_bus(), type, value[0], value[1]])
    dss.circuit_set_active_element(ld)
    column_headings  = ["index", "bus_name", "bus_node_num", "vmag_pu", "all_connected_pce", "all_connected_pde", "over/under voltage", "kW_at_failure", "kvar_at_failure"]
    save_to_excel(log_data, f"..\\violation_log\\{load_to_bus_mapping_ev[ld]}_voltage.xlsx", column_headings)
    log_data = []
    index=0
    for item in line_dict:
        dss.circuit_set_active_element(item)
        for i in range(0, len(line_dict[item]["current"][0])//2, 2):
                if math.sqrt(line_dict[item]["current"][0][i]**2 + line_dict[item]["current"][0][i+1]**2) > dss.lines_read_emerg_amps():
                    log_data.append([index, item, i, math.sqrt(line_dict[item]["current"][0][i]**2 + line_dict[item]["current"][0][i+1]**2), dss.lines_read_bus1, dss.lines_read_bus2, dss.lines_read_emerg_amps()])
                    index+=1
    column_headings  = ["index", "line_name", "line_phase_num", "line_current (in A)", "bus_1", "bus_2", "Max_allowed_amps"]
    save_to_excel(log_data, f"..\\violation_log\\{load_to_bus_mapping_ev[ld]}_current.xlsx", column_headings)

In [12]:
def clear_dir():
    dir_path = "..\\violation_log\\"
    # loop through all files in the directory and delete them
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Error deleting file: {file_path} - {e}")

Now, to come up with a function that is able to 
1. accept a bus
2. place an EV load on it
3. run 1 simulation and get bus voltage violations and line current violations
4. plot said violations on the map


In [13]:
def generate_all_violation_csv(kw, kvar):
    for load in loads:
        if "ev" in load:
            compute_violations(load, ["kw", "kvar"], [kw, kvar])


In [14]:
# def timeseries_load_parse(ld, arr, param):
#     for i in range(len(arr)):
#         change_loads(ld, arr[i])

# load_dict, line_dict, cap_dict, transformer_dict, regulator_dict = make_dicts()
# timeseries_load_parse("Load.671", [i*100 for i in range (1,11)], "kw")
# print(load_dict["Load.671"]["current"])

The following is the practical code, which will read from the csv and save the long and lat as variables of the Bus object.

In [15]:
def read_bus_lats_longs():
    with open('..\\bus_lat_long.csv') as f:
        for line in f:
            dss.circuit_set_active_bus(line.split(",")[0])
            dss.bus_write_latitude(float(line.split(",")[1]))
            dss.bus_write_longitude(float(line.split(",")[2]))

Making a list of all the buses lats and longs

In [16]:
def save_bus_lats_longs():
    bus_lats = []
    bus_longs = []
    for bus in buses:
        dss.circuit_set_active_bus(bus)
        bus_lats.append(dss.bus_read_latitude())
        bus_longs.append(dss.bus_read_longitude())
    return bus_lats, bus_longs

Next section defines what is shown on clicking map elements

In [17]:
def set_popups():
    popup_bus = []
    for bus in buses:
        dss.circuit_set_active_bus(bus)
        data = ",".join(dss.bus_all_pce_active_bus() + dss.bus_all_pde_active_bus())
        popup_bus.append(f"{data}")
    return popup_bus

The following code will be used to generate the required graphs

In [18]:
def map_markers_for_violation(bus):
    v_markers = []
    dataframe_voltage = pd.read_excel(f'..\\violation_log\\{bus}_voltage.xlsx')
    dataframe_current = pd.read_excel(f'..\\violation_log\\{bus}_current.xlsx')
    buses_with_violations = dataframe_voltage["bus_name"].unique()
    lines_with_violations = dataframe_current["line_name"].unique()
    return buses_with_violations, str, lines_with_violations
        

#print(map_markers_for_violation("633"))

In [19]:
def map_everything(bs):
    read_bus_lats_longs()
    bus_lats_longs = save_bus_lats_longs()
    bus_df = pd.DataFrame({'popup_data': set_popups(),
                            'latitude': bus_lats_longs[0],
                            'longitude': bus_lats_longs[1],
                            'icon_num': buses
                        })

    v_violation_coords = []
    print(map_markers_for_violation(bs)[0])
    for bus in map_markers_for_violation(bs)[0]:
        bus = str(bus)
        dss.circuit_set_active_bus(bus)
        v_violation_coords.append([dss.bus_read_latitude(), dss.bus_read_longitude()])

    # won't be showing transformers for now
    # transformer_df = pd.DataFrame({
                        #     'popup_data': popup_trans,
                        #     'latitude': trans_lat,
                        #     'longitude': trans_long,
                        #     'icon_num': [transformers[i].split(".")[-1] for i in range(len(transformers))]
                        # })

    # map centered at Karachi, coordinates from google maps
    m = folium.Map(location=[24.9360971,67.1063258], zoom_start=15)

    # icons using plugins.BeautifyIcon
    for i in bus_df.itertuples():
        folium.Marker(location=[i.latitude, i.longitude],
                    popup=i.popup_data,
                    icon=plugins.BeautifyIcon(number=i.icon_num,
                                                border_color='blue',
                                                border_width=2,
                                                text_color='red',
                                                inner_icon_style='margin-top:0px;')).add_to(m)
        
    for coord in v_violation_coords:
        folium.CircleMarker(location=coord, radius=15, color="red", fill_color="red", fill_opacity=0.1).add_to(m)
        
    # for i in transformer_df.itertuples():
    #     folium.Marker(location=[i.latitude, i.longitude],
    #                   popup=i.popup_data,
    #                   icon=plugins.BeautifyIcon(number=i.icon_num,
    #                                             border_color='blue',
    #                                             border_width=2,
    #                                             text_color='red',
    #                                             inner_icon_style='margin-top:0px;')).add_to(m)

    line_lats_longs = []
    for line in lines:
        dss.circuit_set_active_element(line)
        bus1 = dss.lines_read_bus1()
        bus2 = dss.lines_read_bus2()
        dss.circuit_set_active_bus(bus1)
        bus1_lat = dss.bus_read_latitude()
        bus1_long = dss.bus_read_longitude()
        dss.circuit_set_active_bus(bus2)
        bus2_lat = dss.bus_read_latitude()
        bus2_long = dss.bus_read_longitude()
        line_lats_longs.append([[bus1_lat, bus1_long],[bus2_lat, bus2_long]])

    transformer_connected_buses = []
    transformer_connected_buses.append([[24.936127,67.0905326],[24.941167,67.0894168]])
    transformer_connected_buses.append([[24.9396689,67.084782],[24.933209,67.0854257]])
    transformer_connected_buses.append([[24.933209,67.0854257],[24.930134,67.0866305]])
    # transformer_connected_buses.append([[24.9137492596974, 67.114093381518],[24.921667132445467,67.11482849708392]])
    # transformer_connected_buses.append([[24.91263280670944,67.10864020283294],[24.924105310272832, 67.1137070651443]])
    # transformer_connected_buses.append([[24.924105310272832, 67.1137070651443],[24.91469901870831,67.11791662583576]])

    # add route to map
    for item in line_lats_longs:
        if item in map_markers_for_violation(bs)[-1]:
            plugins.AntPath(item, color="red").add_to(m)
        else:
            plugins.AntPath(item, color="green").add_to(m)
    for item in transformer_connected_buses:
        if item in map_markers_for_violation(bs)[-1]:
            plugins.AntPath(item, color="red").add_to(m)
        else:
            plugins.AntPath(item, color="blue").add_to(m)

    # add tiles to map
    folium.raster_layers.TileLayer('Open Street Map').add_to(m)
    folium.raster_layers.TileLayer('Stamen Terrain').add_to(m)
    folium.raster_layers.TileLayer('Stamen Toner').add_to(m)
    folium.raster_layers.TileLayer('Stamen Watercolor').add_to(m)
    folium.raster_layers.TileLayer('CartoDB Positron').add_to(m)
    folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(m)

    # add layer control to show different maps
    folium.LayerControl().add_to(m)

    # display map    
    return m

In [20]:
load_dict, line_dict, cap_dict, transformer_dict, regulator_dict = make_dicts()
kw = float(input("Please enter the kW requirements of your station: "))
kvar = float(input("Please enter the kVar requirements of your station: "))
# bus_req = input("Please enter the bus at which you would like to place your station: ")
clear_dir()
generate_all_violation_csv(kw, kvar)


In [21]:
bs = input("Please enter the bus at which you would like to place your station: ")
map_everything(bs)


[633 634 671 645 646 692 675 611 652 670 632 680 684]


Testing codes (temp)

In [25]:
# print(load_dict)

For gui

In [23]:
# # Template code

# customtkinter.set_appearance_mode("System")  # Modes: "System" (standard), "Dark", "Light"
# customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"


# class App(customtkinter.CTk):
#     def __init__(self):
#         super().__init__()

#         # configure window
#         self.title("CustomTkinter complex_example.py")
#         self.geometry(f"{1100}x{580}")

#         # configure grid layout (4x4)
#         self.grid_columnconfigure(1, weight=1)
#         self.grid_columnconfigure((2, 3), weight=0)
#         self.grid_rowconfigure((0, 1, 2), weight=1)

#         # create sidebar frame with widgets
#         self.sidebar_frame = customtkinter.CTkFrame(self, width=140, corner_radius=0)
#         self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
#         self.sidebar_frame.grid_rowconfigure(4, weight=1)
#         self.logo_label = customtkinter.CTkLabel(self.sidebar_frame, text="CustomTkinter", font=customtkinter.CTkFont(size=20, weight="bold"))
#         self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))
#         self.sidebar_button_1 = customtkinter.CTkButton(self.sidebar_frame, command=self.sidebar_button_event)
#         self.sidebar_button_1.grid(row=1, column=0, padx=20, pady=10)
#         self.sidebar_button_2 = customtkinter.CTkButton(self.sidebar_frame, command=self.sidebar_button_event)
#         self.sidebar_button_2.grid(row=2, column=0, padx=20, pady=10)
#         self.sidebar_button_3 = customtkinter.CTkButton(self.sidebar_frame, command=self.sidebar_button_event)
#         self.sidebar_button_3.grid(row=3, column=0, padx=20, pady=10)
#         self.appearance_mode_label = customtkinter.CTkLabel(self.sidebar_frame, text="Appearance Mode:", anchor="w")
#         self.appearance_mode_label.grid(row=5, column=0, padx=20, pady=(10, 0))
#         self.appearance_mode_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["Light", "Dark", "System"],
#                                                                        command=self.change_appearance_mode_event)
#         self.appearance_mode_optionemenu.grid(row=6, column=0, padx=20, pady=(10, 10))
#         self.scaling_label = customtkinter.CTkLabel(self.sidebar_frame, text="UI Scaling:", anchor="w")
#         self.scaling_label.grid(row=7, column=0, padx=20, pady=(10, 0))
#         self.scaling_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["80%", "90%", "100%", "110%", "120%"],
#                                                                command=self.change_scaling_event)
#         self.scaling_optionemenu.grid(row=8, column=0, padx=20, pady=(10, 20))

#         # create main entry and button
#         self.entry = customtkinter.CTkEntry(self, placeholder_text="CTkEntry")
#         self.entry.grid(row=3, column=1, columnspan=2, padx=(20, 0), pady=(20, 20), sticky="nsew")

#         self.main_button_1 = customtkinter.CTkButton(master=self, fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"))
#         self.main_button_1.grid(row=3, column=3, padx=(20, 20), pady=(20, 20), sticky="nsew")

#         # create textbox
#         self.textbox = customtkinter.CTkTextbox(self, width=250)
#         self.textbox.grid(row=0, column=1, padx=(20, 0), pady=(20, 0), sticky="nsew")

#         # create tabview
#         self.tabview = customtkinter.CTkTabview(self, width=250)
#         self.tabview.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
#         self.tabview.add("CTkTabview")
#         self.tabview.add("Tab 2")
#         self.tabview.add("Tab 3")
#         self.tabview.tab("CTkTabview").grid_columnconfigure(0, weight=1)  # configure grid of individual tabs
#         self.tabview.tab("Tab 2").grid_columnconfigure(0, weight=1)

#         self.optionmenu_1 = customtkinter.CTkOptionMenu(self.tabview.tab("CTkTabview"), dynamic_resizing=False,
#                                                         values=["Value 1", "Value 2", "Value Long Long Long"])
#         self.optionmenu_1.grid(row=0, column=0, padx=20, pady=(20, 10))
#         self.combobox_1 = customtkinter.CTkComboBox(self.tabview.tab("CTkTabview"),
#                                                     values=["Value 1", "Value 2", "Value Long....."])
#         self.combobox_1.grid(row=1, column=0, padx=20, pady=(10, 10))
#         self.string_input_button = customtkinter.CTkButton(self.tabview.tab("CTkTabview"), text="Open CTkInputDialog",
#                                                            command=self.open_input_dialog_event)
#         self.string_input_button.grid(row=2, column=0, padx=20, pady=(10, 10))
#         self.label_tab_2 = customtkinter.CTkLabel(self.tabview.tab("Tab 2"), text="CTkLabel on Tab 2")
#         self.label_tab_2.grid(row=0, column=0, padx=20, pady=20)

#         # create radiobutton frame
#         self.radiobutton_frame = customtkinter.CTkFrame(self)
#         self.radiobutton_frame.grid(row=0, column=3, padx=(20, 20), pady=(20, 0), sticky="nsew")
#         self.radio_var = tkinter.IntVar(value=0)
#         self.label_radio_group = customtkinter.CTkLabel(master=self.radiobutton_frame, text="CTkRadioButton Group:")
#         self.label_radio_group.grid(row=0, column=2, columnspan=1, padx=10, pady=10, sticky="")
#         self.radio_button_1 = customtkinter.CTkRadioButton(master=self.radiobutton_frame, variable=self.radio_var, value=0)
#         self.radio_button_1.grid(row=1, column=2, pady=10, padx=20, sticky="n")
#         self.radio_button_2 = customtkinter.CTkRadioButton(master=self.radiobutton_frame, variable=self.radio_var, value=1)
#         self.radio_button_2.grid(row=2, column=2, pady=10, padx=20, sticky="n")
#         self.radio_button_3 = customtkinter.CTkRadioButton(master=self.radiobutton_frame, variable=self.radio_var, value=2)
#         self.radio_button_3.grid(row=3, column=2, pady=10, padx=20, sticky="n")

#         #create slider and progressbar frame
#         self.slider_progressbar_frame = customtkinter.CTkFrame(self, fg_color="transparent")
#         self.slider_progressbar_frame.grid(row=1, column=1, padx=(20, 0), pady=(20, 0), sticky="nsew")
#         self.slider_progressbar_frame.grid_columnconfigure(0, weight=1)
#         self.slider_progressbar_frame.grid_rowconfigure(4, weight=1)
#         self.seg_button_1 = customtkinter.CTkSegmentedButton(self.slider_progressbar_frame)
#         self.seg_button_1.grid(row=0, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
#         self.progressbar_1 = customtkinter.CTkProgressBar(self.slider_progressbar_frame)
#         self.progressbar_1.grid(row=1, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
#         self.progressbar_2 = customtkinter.CTkProgressBar(self.slider_progressbar_frame)
#         self.progressbar_2.grid(row=2, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
#         self.slider_1 = customtkinter.CTkSlider(self.slider_progressbar_frame, from_=0, to=1, number_of_steps=4)
#         self.slider_1.grid(row=3, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
#         self.slider_2 = customtkinter.CTkSlider(self.slider_progressbar_frame, orientation="vertical")
#         self.slider_2.grid(row=0, column=1, rowspan=5, padx=(10, 10), pady=(10, 10), sticky="ns")
#         self.progressbar_3 = customtkinter.CTkProgressBar(self.slider_progressbar_frame, orientation="vertical")
#         self.progressbar_3.grid(row=0, column=2, rowspan=5, padx=(10, 20), pady=(10, 10), sticky="ns")

#         # create scrollable frame
#         self.scrollable_frame = customtkinter.CTkScrollableFrame(self, label_text="CTkScrollableFrame")
#         self.scrollable_frame.grid(row=1, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
#         self.scrollable_frame.grid_columnconfigure(0, weight=1)
#         self.scrollable_frame_switches = []
#         for i in range(100):
#             switch = customtkinter.CTkSwitch(master=self.scrollable_frame, text=f"CTkSwitch {i}")
#             switch.grid(row=i, column=0, padx=10, pady=(0, 20))
#             self.scrollable_frame_switches.append(switch)

#         # create checkbox and switch frame
#         self.checkbox_slider_frame = customtkinter.CTkFrame(self)
#         self.checkbox_slider_frame.grid(row=1, column=3, padx=(20, 20), pady=(20, 0), sticky="nsew")
#         self.checkbox_1 = customtkinter.CTkCheckBox(master=self.checkbox_slider_frame)
#         self.checkbox_1.grid(row=1, column=0, pady=(20, 0), padx=20, sticky="n")
#         self.checkbox_2 = customtkinter.CTkCheckBox(master=self.checkbox_slider_frame)
#         self.checkbox_2.grid(row=2, column=0, pady=(20, 0), padx=20, sticky="n")
#         self.checkbox_3 = customtkinter.CTkCheckBox(master=self.checkbox_slider_frame)
#         self.checkbox_3.grid(row=3, column=0, pady=20, padx=20, sticky="n")

#         # set default values
#         self.sidebar_button_3.configure(state="disabled", text="Disabled CTkButton")
#         self.checkbox_3.configure(state="disabled")
#         self.checkbox_1.select()
#         self.scrollable_frame_switches[0].select()
#         self.scrollable_frame_switches[4].select()
#         self.radio_button_3.configure(state="disabled")
#         self.appearance_mode_optionemenu.set("Dark")
#         self.scaling_optionemenu.set("100%")
#         self.optionmenu_1.set("CTkOptionmenu")
#         self.combobox_1.set("CTkComboBox")
#         self.slider_1.configure(command=self.progressbar_2.set)
#         self.slider_2.configure(command=self.progressbar_3.set)
#         self.progressbar_1.configure(mode="indeterminnate")
#         self.progressbar_1.start()
#         self.textbox.insert("0.0", "CTkTextbox\n\n" + "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua.\n\n" * 20)
#         self.seg_button_1.configure(values=["CTkSegmentedButton", "Value 2", "Value 3"])
#         self.seg_button_1.set("Value 2")

#     def open_input_dialog_event(self):
#         dialog = customtkinter.CTkInputDialog(text="Type in a number:", title="CTkInputDialog")
#         print("CTkInputDialog:", dialog.get_input())

#     def change_appearance_mode_event(self, new_appearance_mode: str):
#         customtkinter.set_appearance_mode(new_appearance_mode)

#     def change_scaling_event(self, new_scaling: str):
#         new_scaling_float = int(new_scaling.replace("%", "")) / 100
#         customtkinter.set_widget_scaling(new_scaling_float)

#     def sidebar_button_event(self):
#         print("sidebar_button click")


# if __name__ == "__main__":
#     app = App()
#     app.mainloop()
